<a href="https://colab.research.google.com/github/coding-cosmos/Sluice-Gate-Cavitation-Index/blob/main/notebooks/Sluice_Gate_Cavitation_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Archive


## Data Loading

In [ ]:
import pandas as pd
import optuna
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load and rename columns
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dim.csv")
df.columns = ['AR', 'L', 'phi', 'QW', 'Qa', 'h', 'Va', 'R', 'sigma']


In [ ]:
def split_data(df, target_col='sigma', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [ ]:
df.head()

,AR,L,phi,QW,Qa,h,Va,R,sigma
0,0.6,1,10,1.6,0.000000,0.94,0.00,0.004,19.936623
1,0.6,1,10,3.2,3.467140,0.94,2.18,0.004,4.780871
2,0.6,1,10,4.8,10.846741,0.94,6.82,0.004,1.330924
3,0.6,1,10,6.4,16.683624,0.94,10.49,0.004,0.069292
4,0.6,1,10,8.0,23.458861,0.94,14.75,0.004,-0.691540


## GBM

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
def objective_gbr(trial, X_train, y_train, X_test, y_test):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', [ 'sqrt', 'log2', None])
    }

    model = GradientBoostingRegressor(**params)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


In [ ]:
def tune_and_train_gbr(X_train, y_train, X_test, y_test, n_trials=50):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective_gbr(trial, X_train, y_train, X_test, y_test), n_trials=n_trials)

    print("Best Trial Parameters (GBR):", study.best_trial.params)

    best_params = study.best_trial.params
    model = GradientBoostingRegressor(**best_params)
    model.fit(X_train, y_train)

    return model, study


In [ ]:
def evaluate_split_gbr(df, test_ratio, n_trials=50):
    X_train, X_test, y_train, y_test = split_data(df, test_size=test_ratio)
    model, study = tune_and_train_gbr(X_train, y_train, X_test, y_test, n_trials=n_trials)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"✅ [GBR] Test Ratio: {test_ratio} → Final Test RMSE: {rmse:.4f}")
    return rmse, study.best_trial.params


In [ ]:
def compare_splits_gbr(df, split_list=[0.1, 0.2, 0.3], n_trials=30):
    results = []
    for split in split_list:
        print(f"\n🧪 [GBR] Evaluating Test Split: {split}")
        rmse, params = evaluate_split_gbr(df, test_ratio=split, n_trials=n_trials)
        results.append({'split': split, 'rmse': rmse, 'params': params})
    return pd.DataFrame(results)


In [ ]:
results_gbr = compare_splits_gbr(df, split_list=[0.2, 0.25, 0.3,0.4], n_trials=100)
print("\n📈 [GBR] Results Summary:")
print(results_gbr[['split', 'rmse']])


[I 2025-06-09 16:43:35,873] A new study created in memory with name: no-name-71077bcc-3440-465b-9e15-90f7b78a70d4



🧪 [GBR] Evaluating Test Split: 0.2


[I 2025-06-09 16:43:36,246] Trial 0 finished with value: 0.5928612373697129 and parameters: {'n_estimators': 185, 'max_depth': 7, 'learning_rate': 0.13695261080212182, 'subsample': 0.8597076714084161, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 0 with value: 0.5928612373697129.
[I 2025-06-09 16:43:36,652] Trial 1 finished with value: 10.406254812509275 and parameters: {'n_estimators': 369, 'max_depth': 5, 'learning_rate': 0.002973949598197249, 'subsample': 0.7092871272327844, 'min_samples_split': 2, 'min_samples_leaf': 14, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.5928612373697129.
[I 2025-06-09 16:43:36,972] Trial 2 finished with value: 1.4955060785277103 and parameters: {'n_estimators': 218, 'max_depth': 5, 'learning_rate': 0.2530625322255241, 'subsample': 0.7649814593835482, 'min_samples_split': 4, 'min_samples_leaf': 19, 'max_features': None}. Best is trial 0 with value: 0.5928612373697129.
[I 2025-06-09 16:43:37,150] Trial 3 fi

Best Trial Parameters (GBR): {'n_estimators': 488, 'max_depth': 9, 'learning_rate': 0.030526105174335707, 'subsample': 0.9118171027555896, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': None}


[I 2025-06-09 16:44:57,850] A new study created in memory with name: no-name-9ee5726f-e2b3-4ba9-9b11-647690e601c0


...

Best Trial Parameters (GBR): {'n_estimators': 104, 'max_depth': 7, 'learning_rate': 0.12302867401017932, 'subsample': 0.991251750051151, 'min_samples_split': 13, 'min_samples_leaf': 2, 'max_features': None}


[I 2025-06-09 16:46:33,349] A new study created in memory with name: no-name-f66df0f5-952d-49fa-9ac2-f1a5f428cbac


✅ [GBR] Test Ratio: 0.3 → Final Test RMSE: 0.8250

🧪 [GBR] Evaluating Test Split: 0.4


[I 2025-06-09 16:46:33,936] Trial 0 finished with value: 11.501557039522927 and parameters: {'n_estimators': 264, 'max_depth': 4, 'learning_rate': 0.003933446160576122, 'subsample': 0.8859707776132849, 'min_samples_split': 14, 'min_samples_leaf': 13, 'max_features': 'log2'}. Best is trial 0 with value: 11.501557039522927.
[I 2025-06-09 16:46:34,836] Trial 1 finished with value: 16.287532732684948 and parameters: {'n_estimators': 358, 'max_depth': 9, 'learning_rate': 0.0007083165570817755, 'subsample': 0.8707127436397735, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 0 with value: 11.501557039522927.
[I 2025-06-09 16:46:35,388] Trial 2 finished with value: 2.83108903179839 and parameters: {'n_estimators': 299, 'max_depth': 2, 'learning_rate': 0.06781607583827615, 'subsample': 0.5066150389014852, 'min_samples_split': 6, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 2 with value: 2.83108903179839.
[I 2025-06-09 16:46:36,162] Trial 3

Best Trial Parameters (GBR): {'n_estimators': 487, 'max_depth': 9, 'learning_rate': 0.019028953389915445, 'subsample': 0.9265276748450405, 'min_samples_split': 16, 'min_samples_leaf': 1, 'max_features': None}
✅ [GBR] Test Ratio: 0.4 → Final Test RMSE: 1.0112

📈 [GBR] Results Summary:
   split      rmse
0   0.20  0.410044
1   0.25  0.579292
2   0.30  0.825048
3   0.40  1.011173


## 📈 Gradient Boosting Regressor (GBR) Results Summary

| Split Ratio |   RMSE    |
|-------------|-----------|
| 0.20        | 0.387702  |
| 0.25        | 0.625173  |
| 0.30        | 0.943514  |
| 0.40        | 1.126326  |


## XGBoost

In [ ]:
!pip install xgboost --quiet


In [ ]:
from xgboost import XGBRegressor

In [ ]:
def objective_xgb(trial, X_train, y_train, X_test, y_test):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = XGBRegressor(
        **params,
        objective='reg:squarederror',
        verbosity=0,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


In [ ]:
def tune_and_train_xgb(X_train, y_train, X_test, y_test, n_trials=50):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective_xgb(trial, X_train, y_train, X_test, y_test), n_trials=n_trials)

    print("Best Trial Parameters (XGB):", study.best_trial.params)

    best_params = study.best_trial.params
    model = XGBRegressor(
        **best_params,
        objective='reg:squarederror',
        verbosity=0,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    return model, study


In [ ]:
def evaluate_split_xgb(df, test_ratio, n_trials=50):
    X_train, X_test, y_train, y_test = split_data(df, test_size=test_ratio)
    model, study = tune_and_train_xgb(X_train, y_train, X_test, y_test, n_trials=n_trials)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"✅ [XGB] Test Ratio: {test_ratio} → Final Test RMSE: {rmse:.4f}")
    return rmse, study.best_trial.params


In [ ]:
def compare_splits_xgb(df, split_list=[0.1, 0.2, 0.3], n_trials=30):
    results = []
    for split in split_list:
        print(f"\n🧪 [XGB] Evaluating Test Split: {split}")
        rmse, params = evaluate_split_xgb(df, test_ratio=split, n_trials=n_trials)
        results.append({'split': split, 'rmse': rmse, 'params': params})
    return pd.DataFrame(results)


In [ ]:
results_xgb = compare_splits_xgb(df, split_list=[0.2, 0.25, 0.3,0.4], n_trials=100)
print("\n📈 [XGB] Results Summary:")
print(results_xgb[['split', 'rmse']])


[I 2025-06-10 12:56:01,852] A new study created in memory with name: no-name-69bb5bb3-9cbc-46f5-8a10-1d36cd0d8aca



🧪 [XGB] Evaluating Test Split: 0.2


[I 2025-06-10 12:56:02,095] Trial 0 finished with value: 27.98739403822205 and parameters: {'n_estimators': 274, 'max_depth': 4, 'learning_rate': 0.00013418906719344425, 'subsample': 0.5516185882002248, 'colsample_bytree': 0.9300272983550875, 'gamma': 4.752687211549028, 'reg_alpha': 3.52168427381472, 'reg_lambda': 1.4525726105725938}. Best is trial 0 with value: 27.98739403822205.
[I 2025-06-10 12:56:02,201] Trial 1 finished with value: 25.450116906864576 and parameters: {'n_estimators': 131, 'max_depth': 3, 'learning_rate': 0.0012997243439984432, 'subsample': 0.6355282246747402, 'colsample_bytree': 0.5510344194220751, 'gamma': 1.8978204795702762, 'reg_alpha': 1.2941510080495218, 'reg_lambda': 0.8989255367176602}. Best is trial 1 with value: 25.450116906864576.
[I 2025-06-10 12:56:03,350] Trial 2 finished with value: 2.3782205371238914 and parameters: {'n_estimators': 133, 'max_depth': 5, 'learning_rate': 0.2191048703917789, 'subsample': 0.6153746210283686, 'colsample_bytree': 0.570836

KeyboardInterrupt: 

## 📈 XGBoost Regressor (XGB) Results Summary

| Split Ratio |   RMSE    |
|-------------|-----------|
| 0.20        | 0.876111  |
| 0.25        | 0.874201  |
| 0.30        | 0.818592  |
| 0.40        | 1.216925  |


## Catboost

In [ ]:
!pip install catboost --quiet


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
def objective_catboost(trial, X_train, y_train, X_test, y_test):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0)
    }

    model = CatBoostRegressor(
        **params,
        loss_function='RMSE',
        verbose=0,
        random_seed=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


In [ ]:
def tune_and_train_catboost(X_train, y_train, X_test, y_test, n_trials=50):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective_catboost(trial, X_train, y_train, X_test, y_test), n_trials=n_trials)

    print("Best Trial Parameters (CatBoost):", study.best_trial.params)

    best_params = study.best_trial.params
    model = CatBoostRegressor(
        **best_params,
        loss_function='RMSE',
        verbose=0,
        random_seed=42
    )
    model.fit(X_train, y_train)

    return model, study


In [ ]:
def evaluate_split_catboost(df, test_ratio, n_trials=50):
    X_train, X_test, y_train, y_test = split_data(df, test_size=test_ratio)
    model, study = tune_and_train_catboost(X_train, y_train, X_test, y_test, n_trials=n_trials)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"✅ [CatBoost] Test Ratio: {test_ratio} → Final Test RMSE: {rmse:.4f}")
    return rmse, study.best_trial.params


In [ ]:
def compare_splits_catboost(df, split_list=[0.1, 0.2, 0.3], n_trials=30):
    results = []
    for split in split_list:
        print(f"\n🧪 [CatBoost] Evaluating Test Split: {split}")
        rmse, params = evaluate_split_catboost(df, test_ratio=split, n_trials=n_trials)
        results.append({'split': split, 'rmse': rmse, 'params': params})
    return pd.DataFrame(results)


In [ ]:
results_catboost = compare_splits_catboost(df, split_list=[0.2, 0.25, 0.3,0.4], n_trials=100)
print("\n📈 [CatBoost] Results Summary:")
print(results_catboost[['split', 'rmse']])


## 📈 CatBoost Regressor Results Summary

| Split Ratio |   RMSE    |
|-------------|-----------|
| 0.20        | 0.370242  |
| 0.25        | 1.242720  |
| 0.30        | 0.904113  |
| 0.40        | 1.131831  |


## XRT

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor

In [ ]:
def objective_xrt(trial, X_train, y_train, X_test, y_test):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = ExtraTreesRegressor(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


In [ ]:
def tune_and_train_xrt(X_train, y_train, X_test, y_test, n_trials=50):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective_xrt(trial, X_train, y_train, X_test, y_test), n_trials=n_trials)

    print("Best Trial Parameters (XRT):", study.best_trial.params)

    model = ExtraTreesRegressor(**study.best_trial.params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    return model, study


In [ ]:
def evaluate_split_xrt(df, test_ratio, n_trials=50):
    X_train, X_test, y_train, y_test = split_data(df, test_size=test_ratio)
    model, study = tune_and_train_xrt(X_train, y_train, X_test, y_test, n_trials=n_trials)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"✅ [XRT] Test Ratio: {test_ratio} → Final Test RMSE: {rmse:.4f}")
    return rmse, study.best_trial.params


In [ ]:
def compare_splits_xrt(df, split_list=[0.1, 0.2, 0.3], n_trials=30):
    results = []
    for split in split_list:
        print(f"\n🧪 [XRT] Evaluating Test Split: {split}")
        rmse, params = evaluate_split_xrt(df, test_ratio=split, n_trials=n_trials)
        results.append({'split': split, 'rmse': rmse, 'params': params})
    return pd.DataFrame(results)


In [ ]:
results_xrt = compare_splits_xrt(df, split_list=[0.2, 0.25,.3,.4], n_trials=100)
print("\n📊 XRT Results:\n", results_xrt[['split', 'rmse']])

## 📊 Extra Trees Regressor (XRT) Results Summary

| Split Ratio |   RMSE    |
|-------------|-----------|
| 0.20        | 2.178530  |
| 0.25        | 2.270443  |
| 0.30        | 1.921216  |
| 0.40        | 1.839868  |


## Random Forest

In [ ]:
def objective_rf(trial, X_train, y_train, X_test, y_test):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = RandomForestRegressor(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


In [ ]:
def tune_and_train_rf(X_train, y_train, X_test, y_test, n_trials=50):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective_rf(trial, X_train, y_train, X_test, y_test), n_trials=n_trials)

    print("Best Trial Parameters (RF):", study.best_trial.params)

    model = RandomForestRegressor(**study.best_trial.params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    return model, study


In [ ]:
def evaluate_split_rf(df, test_ratio, n_trials=50):
    X_train, X_test, y_train, y_test = split_data(df, test_size=test_ratio)
    model, study = tune_and_train_rf(X_train, y_train, X_test, y_test, n_trials=n_trials)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    print(f"✅ [RF] Test Ratio: {test_ratio} → Final Test RMSE: {rmse:.4f}")
    return rmse, study.best_trial.params


In [ ]:
def compare_splits_rf(df, split_list=[0.1, 0.2, 0.3], n_trials=30):
    results = []
    for split in split_list:
        print(f"\n🧪 [RF] Evaluating Test Split: {split}")
        rmse, params = evaluate_split_rf(df, test_ratio=split, n_trials=n_trials)
        results.append({'split': split, 'rmse': rmse, 'params': params})
    return pd.DataFrame(results)


In [ ]:
results_rf = compare_splits_rf(df, split_list=[0.2, 0.25,0.3,0.4], n_trials=100)
print("\n📊 RF Results:\n", results_rf[['split', 'rmse']])

## 📊 Random Forest Regressor (RF) Results Summary

| Split Ratio |   RMSE    |
|-------------|-----------|
| 0.20        | 1.065148  |
| 0.25        | 1.733187  |
| 0.30        | 1.749576  |
| 0.40        | 2.334590  |


## Plots


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_feature_importance(model, feature_names=None, top_n=20, title="Feature Importance"):
    # Fallback feature names
    if feature_names is None:
        feature_names = [f"Feature {i}" for i in range(len(model.feature_importances_))]

    importances = model.feature_importances_
    indices = np.argsort(importances)[-top_n:][::-1]  # Top-N important features

    plt.figure(figsize=(10, 6))
    plt.barh(range(top_n), importances[indices], align='center')
    plt.yticks(range(top_n), [feature_names[i] for i in indices])
    plt.xlabel("Importance Score")
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


# Training


## Utils

In [ ]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.1 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import optuna.visualization as vis

In [ ]:
import pandas as pd
import numpy as np


# Load and rename columns
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dim.csv")
df.columns = ['AR', 'phi', 'L', 'QW', 'Qa', 'Va', 'h', 'R', 'sigma']
X = df.drop(columns=['sigma'])
y = df['sigma']


In [ ]:
df

,AR,phi,L,QW,Qa,Va,h,R,sigma
0,0.6,10,1,0.95,0.000000,0.00,0.9400,0.004000,56.551530
1,0.6,10,1,1.90,4.325973,2.72,0.9400,0.004000,13.240200
2,0.6,10,1,2.85,9.956100,6.26,0.9400,0.004000,4.170252
3,0.6,10,1,3.80,18.576237,11.68,0.9400,0.004000,-0.603721
4,0.6,10,1,4.75,25.335570,15.93,0.9400,0.004000,-2.664416
...,...,...,...,...,...,...,...,...,...
499,1.0,60,6,12.60,2.624212,1.65,7.9886,0.012535,11.593609
500,1.0,60,6,16.80,4.739485,2.98,7.9886,0.012535,6.168513
501,1.0,60,6,21.00,6.377629,4.01,7.9886,0.012535,3.683787
502,1.0,60,6,25.20,8.890511,5.59,7.9886,0.012535,2.171858


In [ ]:
def split_data(df, target_col='sigma', test_size=0.2, random_state=42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [ ]:
def train_model(model_cls, X, y, split_ratio=0.2, tune_func=None, model_name="Model", n_trials=30, **kwargs):
    """
    Trains any model with optional Optuna tuning.

    Args:
        model_cls: A model class like GradientBoostingRegressor
        X: Feature dataframe
        y: Target series
        split_ratio: Float ratio for test set
        tune_func: A function accepting (trial, X_train, y_train, X_test, y_test) → rmse
        model_name: Name for logging
        n_trials: Optuna tuning trials (only used if tune_func is provided)
        **kwargs: Default model kwargs if no tuning is done

    Returns:
        model, rmse, y_pred, y_test
    """

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split_ratio, random_state=42)

    if tune_func:
        study = optuna.create_study(direction="minimize")
        study.optimize(lambda trial: tune_func(trial, X_train, y_train, X_test, y_test, model_cls), n_trials=n_trials)
        best_params = study.best_params
        print(f"✅ Best params for {model_name}: {best_params}")
        model = model_cls(**best_params)

        fig = vis.plot_optimization_history(study)
        fig.show()
    else:
        model = model_cls(**kwargs)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"📊 {model_name} RMSE: {rmse:.4f}")

    return {
    "model": model,
    "rmse": rmse,
    "X_train": X_train,
    "X_test": X_test,
    "y_train": y_train,
    "y_test": y_test,
    "y_pred": y_pred
    }



## GBM

In [ ]:
def objective_gbm(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'max_features': trial.suggest_categorical('max_features', [ 'sqrt', 'log2', None])
    }

    model = model_cls(**params)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse

### 80-20 Split

In [ ]:
gbm_20 = train_model(GradientBoostingRegressor,X,y,0.2,objective_gbm,"GBM",n_trials=500)

[I 2025-06-10 10:17:36,315] A new study created in memory with name: no-name-09b14935-4e99-41f1-a0d4-24b95d879586
[I 2025-06-10 10:17:36,444] Trial 0 finished with value: 27.627542992299507 and parameters: {'n_estimators': 141, 'max_depth': 9, 'learning_rate': 0.00033879619444447136, 'subsample': 0.9069163784247223, 'min_samples_split': 18, 'min_samples_leaf': 6, 'max_features': 'log2'}. Best is trial 0 with value: 27.627542992299507.
[I 2025-06-10 10:17:36,718] Trial 1 finished with value: 2.534974271314407 and parameters: {'n_estimators': 415, 'max_depth': 4, 'learning_rate': 0.20455496260758424, 'subsample': 0.6800709485477614, 'min_samples_split': 18, 'min_samples_leaf': 9, 'max_features': 'sqrt'}. Best is trial 1 with value: 2.534974271314407.
[I 2025-06-10 10:17:36,888] Trial 2 finished with value: 23.70070254701778 and parameters: {'n_estimators': 158, 'max_depth': 6, 'learning_rate': 0.001464150728878486, 'subsample': 0.7884485486264696, 'min_samples_split': 12, 'min_samples_le

✅ Best params for GBM: {'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.10664589740085356, 'subsample': 0.6623650248841277, 'min_samples_split': 17, 'min_samples_leaf': 2, 'max_features': None}


📊 GBM RMSE: 1.2713


✅ Best params for GBM: {'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.10664589740085356, 'subsample': 0.6623650248841277, 'min_samples_split': 17, 'min_samples_leaf': 2, 'max_features': None}


In [ ]:
params_gbm_20 = {'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.10664589740085356, 'subsample': 0.6623650248841277, 'min_samples_split': 17, 'min_samples_leaf': 2, 'max_features': None}

In [ ]:
gbm_20 = train_model(GradientBoostingRegressor,X,y,0.2,None,"GBM",n_trials=0,**params_gbm_20)

📊 GBM RMSE: 1.2323


In [ ]:
print(gbm_20['rmse'])
print(r2_score(gbm_20['y_test'],gbm_20['y_pred']))

1.232313567103787
0.9981712649003258


In [ ]:
df_train = gbm_20['X_train'].copy()
df_test = gbm_20['X_test'].copy()

df_train['sigma'] = gbm_20['y_train']
df_test['sigma'] = gbm_20['y_test']

In [ ]:
df_test['gbm'] = gbm_20['y_pred']
df_train['gbm'] = gbm_20['model'].predict(gbm_20['X_train'])

In [ ]:
params_gbm_20 = {'n_estimators': 310, 'max_depth': 7, 'learning_rate': 0.10664589740085356, 'subsample': 0.6623650248841277, 'min_samples_split': 17, 'min_samples_leaf': 2, 'max_features': None}
gbm_20 = train_model(GradientBoostingRegressor,X,y,0.2,None,"GBM",n_trials=0,**params_gbm_20)
df_test['gbm'] = gbm_20['y_pred']
df_train['gbm'] = gbm_20['model'].predict(gbm_20['X_train'])

### 75-25 Split

In [ ]:
gbm_25 = train_model(GradientBoostingRegressor,X,y,0.25,objective_gbm,"GBM",n_trials=500)

[I 2025-06-10 10:21:26,975] A new study created in memory with name: no-name-fd3b0a49-c045-4d53-9475-376412a2f863
[I 2025-06-10 10:21:27,195] Trial 0 finished with value: 22.016889042381067 and parameters: {'n_estimators': 316, 'max_depth': 8, 'learning_rate': 0.0012245926890711416, 'subsample': 0.8615651123040013, 'min_samples_split': 2, 'min_samples_leaf': 17, 'max_features': 'sqrt'}. Best is trial 0 with value: 22.016889042381067.
[I 2025-06-10 10:21:27,450] Trial 1 finished with value: 27.29475188077529 and parameters: {'n_estimators': 396, 'max_depth': 5, 'learning_rate': 0.00019438952181880892, 'subsample': 0.5443317211013914, 'min_samples_split': 16, 'min_samples_leaf': 11, 'max_features': 'sqrt'}. Best is trial 0 with value: 22.016889042381067.
[I 2025-06-10 10:21:27,528] Trial 2 finished with value: 4.05469763934922 and parameters: {'n_estimators': 103, 'max_depth': 5, 'learning_rate': 0.050673157138370194, 'subsample': 0.6589103787788974, 'min_samples_split': 12, 'min_samples

✅ Best params for GBM: {'n_estimators': 453, 'max_depth': 9, 'learning_rate': 0.104443844901846, 'subsample': 0.9999731682400737, 'min_samples_split': 12, 'min_samples_leaf': 2, 'max_features': None}


📊 GBM RMSE: 1.6497


In [ ]:
print(gbm_25['rmse'])
print(r2_score(gbm_25['y_test'],gbm_25['y_pred']))

1.649744727194187
0.9967167526906859


### 70-30 Split

In [ ]:
gbm_30 = train_model(GradientBoostingRegressor,X,y,0.3,objective_gbm,"GBM",n_trials=500)

[I 2025-06-10 10:27:05,042] A new study created in memory with name: no-name-135ad4d7-39f6-4e55-a954-38bdd2e1aecd
[I 2025-06-10 10:27:05,363] Trial 0 finished with value: 24.33268694630708 and parameters: {'n_estimators': 297, 'max_depth': 4, 'learning_rate': 0.0007320702588884475, 'subsample': 0.5581718231235067, 'min_samples_split': 17, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 24.33268694630708.
[I 2025-06-10 10:27:05,769] Trial 1 finished with value: 4.603346570732267 and parameters: {'n_estimators': 335, 'max_depth': 3, 'learning_rate': 0.02799404698747795, 'subsample': 0.8977012682045158, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 1 with value: 4.603346570732267.
[I 2025-06-10 10:27:05,871] Trial 2 finished with value: 15.953997842558687 and parameters: {'n_estimators': 77, 'max_depth': 7, 'learning_rate': 0.00895180189112321, 'subsample': 0.7600755122043918, 'min_samples_split': 11, 'min_samples_leaf': 2, '

✅ Best params for GBM: {'n_estimators': 120, 'max_depth': 8, 'learning_rate': 0.061506499435021365, 'subsample': 0.956768843627744, 'min_samples_split': 11, 'min_samples_leaf': 2, 'max_features': None}


📊 GBM RMSE: 1.6785


In [ ]:
print(gbm_30['rmse'])
print(r2_score(gbm_30['y_test'],gbm_30['y_pred']))

1.6784763162243512
0.9967468443381806


### 60-40 Split

In [ ]:
gbm_40 = train_model(GradientBoostingRegressor,X,y,0.4,objective_gbm,"GBM",n_trials=500)

[I 2025-06-10 10:36:44,986] A new study created in memory with name: no-name-3a172dd5-7688-4621-86de-dbe0e024582b
[I 2025-06-10 10:36:45,158] Trial 0 finished with value: 5.129725702771693 and parameters: {'n_estimators': 267, 'max_depth': 5, 'learning_rate': 0.2980459608819221, 'subsample': 0.683944983324835, 'min_samples_split': 9, 'min_samples_leaf': 17, 'max_features': 'sqrt'}. Best is trial 0 with value: 5.129725702771693.
[I 2025-06-10 10:36:45,274] Trial 1 finished with value: 28.16796546514671 and parameters: {'n_estimators': 154, 'max_depth': 9, 'learning_rate': 0.0003435324898054927, 'subsample': 0.5286657485869816, 'min_samples_split': 20, 'min_samples_leaf': 15, 'max_features': None}. Best is trial 0 with value: 5.129725702771693.
[I 2025-06-10 10:36:45,398] Trial 2 finished with value: 27.154470614704188 and parameters: {'n_estimators': 174, 'max_depth': 8, 'learning_rate': 0.0005431319759765579, 'subsample': 0.505206254241262, 'min_samples_split': 2, 'min_samples_leaf': 6

✅ Best params for GBM: {'n_estimators': 433, 'max_depth': 7, 'learning_rate': 0.19103378722815845, 'subsample': 0.9183746560471913, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': None}


📊 GBM RMSE: 2.7456


In [ ]:
print(gbm_40['rmse'])
print(r2_score(gbm_40['y_test'],gbm_40['y_pred']))

2.7455530089192495
0.9910692555439792


## XGBoost

In [ ]:
def objective_xgb(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = model_cls(
        **params,
        objective='reg:squarederror',
        verbosity=0,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20

In [ ]:
xgb_20 = train_model(XGBRegressor,X,y,0.2,objective_xgb,"XGBoost",n_trials=500)

[I 2025-06-10 10:46:06,389] A new study created in memory with name: no-name-8c5b2ddd-0adb-4978-8d55-7520e18951e9
[I 2025-06-10 10:46:07,020] Trial 0 finished with value: 2.890714817890649 and parameters: {'n_estimators': 204, 'max_depth': 6, 'learning_rate': 0.03274687730101624, 'subsample': 0.591705768655254, 'colsample_bytree': 0.8195779441407449, 'gamma': 1.556752269643592, 'reg_alpha': 3.0604890901260062, 'reg_lambda': 2.877659993683288}. Best is trial 0 with value: 2.890714817890649.
[I 2025-06-10 10:46:07,695] Trial 1 finished with value: 10.854525639409987 and parameters: {'n_estimators': 154, 'max_depth': 5, 'learning_rate': 0.00800161480270665, 'subsample': 0.7834777494095264, 'colsample_bytree': 0.6627401937367292, 'gamma': 2.531577987155658, 'reg_alpha': 0.3375646125094478, 'reg_lambda': 1.083011060260407}. Best is trial 0 with value: 2.890714817890649.
[I 2025-06-10 10:46:07,997] Trial 2 finished with value: 17.14161328135656 and parameters: {'n_estimators': 407, 'max_dept

✅ Best params for XGBoost: {'n_estimators': 90, 'max_depth': 7, 'learning_rate': 0.2197171170093579, 'subsample': 0.7962931012626785, 'colsample_bytree': 0.9205899728102821, 'gamma': 0.746117386065956, 'reg_alpha': 0.5906239428833475, 'reg_lambda': 0.09608894620402031}


📊 XGBoost RMSE: 2.7851


✅ Best params for XGBoost: {'n_estimators': 90, 'max_depth': 7, 'learning_rate': 0.2197171170093579, 'subsample': 0.7962931012626785, 'colsample_bytree': 0.9205899728102821, 'gamma': 0.746117386065956, 'reg_alpha': 0.5906239428833475, 'reg_lambda': 0.09608894620402031}


In [ ]:
params_xgb_20 = {'n_estimators': 90, 'max_depth': 7, 'learning_rate': 0.2197171170093579, 'subsample': 0.7962931012626785, 'colsample_bytree': 0.9205899728102821, 'gamma': 0.746117386065956, 'reg_alpha': 0.5906239428833475, 'reg_lambda': 0.09608894620402031}
xgb_20 = train_model(XGBRegressor,X,y,0.2,None,"XGBoost",n_trials=0,**params_xgb_20)
df_test['xgb'] = xgb_20['y_pred']
df_train['xgb'] = xgb_20['model'].predict(xgb_20['X_train'])

📊 XGBoost RMSE: 2.7851


In [ ]:
print(xgb_20['rmse'])
print(r2_score(xgb_20['y_test'],xgb_20['y_pred']))

2.7851248937856257
0.9906589071058471


### 75-25

In [ ]:
xgb_25 = train_model(XGBRegressor,X,y,0.25,objective_xgb,"XGBoost",n_trials=500)

[I 2025-06-10 10:47:37,817] A new study created in memory with name: no-name-362ea4d5-3dbe-41a6-b0ba-46ed6cbc3167
[I 2025-06-10 10:47:37,865] Trial 0 finished with value: 8.846754795023529 and parameters: {'n_estimators': 97, 'max_depth': 9, 'learning_rate': 0.02555670229372364, 'subsample': 0.6413412579148674, 'colsample_bytree': 0.522806185156723, 'gamma': 2.5108456676513864, 'reg_alpha': 2.8158036013499848, 'reg_lambda': 4.474628395992036}. Best is trial 0 with value: 8.846754795023529.
[I 2025-06-10 10:47:38,021] Trial 1 finished with value: 25.287016977874746 and parameters: {'n_estimators': 436, 'max_depth': 7, 'learning_rate': 0.00038686195696531207, 'subsample': 0.9877166047783241, 'colsample_bytree': 0.8924514774982566, 'gamma': 2.4410152205879037, 'reg_alpha': 1.2088786261148772, 'reg_lambda': 4.106587746325158}. Best is trial 0 with value: 8.846754795023529.
[I 2025-06-10 10:47:38,093] Trial 2 finished with value: 2.822719674057909 and parameters: {'n_estimators': 152, 'max_

✅ Best params for XGBoost: {'n_estimators': 68, 'max_depth': 9, 'learning_rate': 0.2980729799188844, 'subsample': 0.8762410657350859, 'colsample_bytree': 0.7770689761903219, 'gamma': 2.746422206579382, 'reg_alpha': 3.2681021485485493, 'reg_lambda': 2.0564014059555893}


📊 XGBoost RMSE: 2.0545


In [ ]:
print(xgb_25['rmse'])
print(r2_score(xgb_25['y_test'],xgb_25['y_pred']))

2.0544949316777927
0.9949080955262117


### 70-30

In [ ]:
xgb_30 = train_model(XGBRegressor,X,y,0.3,objective_xgb,"XGBoost",n_trials=100)

[I 2025-06-10 10:48:32,935] A new study created in memory with name: no-name-952c1853-480c-4203-8195-acb9be661ed1
[I 2025-06-10 10:48:32,979] Trial 0 finished with value: 26.864859578727746 and parameters: {'n_estimators': 90, 'max_depth': 7, 'learning_rate': 0.0013198972504820713, 'subsample': 0.7969958968382438, 'colsample_bytree': 0.9532885607452191, 'gamma': 4.087692242200407, 'reg_alpha': 2.8675871820828522, 'reg_lambda': 1.3790377901643143}. Best is trial 0 with value: 26.864859578727746.
[I 2025-06-10 10:48:33,038] Trial 1 finished with value: 4.957432781665323 and parameters: {'n_estimators': 253, 'max_depth': 3, 'learning_rate': 0.04163641502523041, 'subsample': 0.5450367644801581, 'colsample_bytree': 0.7430998745518562, 'gamma': 3.6128241566663304, 'reg_alpha': 4.374440404420538, 'reg_lambda': 0.7228671843916962}. Best is trial 1 with value: 4.957432781665323.
[I 2025-06-10 10:48:33,132] Trial 2 finished with value: 16.801025695642082 and parameters: {'n_estimators': 249, 'ma

✅ Best params for XGBoost: {'n_estimators': 374, 'max_depth': 8, 'learning_rate': 0.2966137827476755, 'subsample': 0.9109263975602332, 'colsample_bytree': 0.8902949931148685, 'gamma': 0.6854581079758156, 'reg_alpha': 1.0249927282614568, 'reg_lambda': 0.1005881957008662}


📊 XGBoost RMSE: 5.6478


In [ ]:
print(xgb_30['rmse'])
print(r2_score(xgb_30['y_test'],xgb_30['y_pred']))

5.647759389483977
0.9631678744905426


### 60-40

In [ ]:
xgb_40 = train_model(XGBRegressor,X,y,0.4,objective_xgb,"XGBoost",n_trials=500)

[I 2025-06-10 10:48:44,817] A new study created in memory with name: no-name-1855de44-76fd-45b3-8292-fe6ac95b67d3
[I 2025-06-10 10:48:44,918] Trial 0 finished with value: 24.822591728266737 and parameters: {'n_estimators': 293, 'max_depth': 8, 'learning_rate': 0.0008514627676065028, 'subsample': 0.8330083318637254, 'colsample_bytree': 0.5773872418695349, 'gamma': 1.1520433166636723, 'reg_alpha': 2.194667233059151, 'reg_lambda': 2.0074432662495227}. Best is trial 0 with value: 24.822591728266737.
[I 2025-06-10 10:48:44,998] Trial 1 finished with value: 26.35624759999884 and parameters: {'n_estimators': 205, 'max_depth': 6, 'learning_rate': 0.0006149569313755404, 'subsample': 0.7981376619392975, 'colsample_bytree': 0.9891681463880121, 'gamma': 3.74443415334167, 'reg_alpha': 2.217803714016486, 'reg_lambda': 0.8530732765921828}. Best is trial 0 with value: 24.822591728266737.
[I 2025-06-10 10:48:45,153] Trial 2 finished with value: 23.3892822489826 and parameters: {'n_estimators': 475, 'ma

✅ Best params for XGBoost: {'n_estimators': 195, 'max_depth': 7, 'learning_rate': 0.29846931356991735, 'subsample': 0.9316015452034682, 'colsample_bytree': 0.9749397520976734, 'gamma': 0.4003587668831996, 'reg_alpha': 2.3878963615374196, 'reg_lambda': 0.0038829808646893334}


📊 XGBoost RMSE: 5.8929


In [ ]:
print(xgb_40['rmse'])
print(r2_score(xgb_40['y_test'],xgb_40['y_pred']))

5.892859944351598
0.9588584892721745


## CatBoost

In [ ]:
def objective_catb(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0)
    }

    model = model_cls(
        **params,
        loss_function='RMSE',
        verbose=0,
        random_seed=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20

In [ ]:
catb_20 = train_model(CatBoostRegressor,X,y,0.2,objective_catb,"CatBoost",n_trials=500)

[I 2025-06-10 10:57:12,559] A new study created in memory with name: no-name-885ecdcc-a2f8-42f4-ae27-f6c180e104a6
[I 2025-06-10 10:57:15,460] Trial 0 finished with value: 1.4743847114294613 and parameters: {'iterations': 162, 'depth': 10, 'learning_rate': 0.05277219441276904, 'l2_leaf_reg': 5.297027762116596, 'bagging_temperature': 0.01958693857746252, 'random_strength': 0.04496971306609787}. Best is trial 0 with value: 1.4743847114294613.
[I 2025-06-10 10:57:15,770] Trial 1 finished with value: 26.67971859588661 and parameters: {'iterations': 106, 'depth': 7, 'learning_rate': 0.0010977530697622512, 'l2_leaf_reg': 3.0025886986469468, 'bagging_temperature': 0.7938871220142532, 'random_strength': 0.92895396414641}. Best is trial 0 with value: 1.4743847114294613.
[I 2025-06-10 10:57:16,732] Trial 2 finished with value: 18.40554285998957 and parameters: {'iterations': 341, 'depth': 7, 'learning_rate': 0.002509030002466896, 'l2_leaf_reg': 9.834993777573285, 'bagging_temperature': 0.85713812

✅ Best params for CatBoost: {'iterations': 411, 'depth': 7, 'learning_rate': 0.11693909733978508, 'l2_leaf_reg': 1.0058943176207062, 'bagging_temperature': 0.7947457289044807, 'random_strength': 0.02570983831467845}


0:	learn: 23.0829200	total: 1.12ms	remaining: 460ms
1:	learn: 20.7107634	total: 2.11ms	remaining: 432ms
2:	learn: 18.7012581	total: 3.04ms	remaining: 413ms
3:	learn: 16.7208480	total: 3.98ms	remaining: 405ms
4:	learn: 15.1583005	total: 4.96ms	remaining: 402ms
5:	learn: 13.6169841	total: 5.92ms	remaining: 400ms
6:	learn: 12.2274416	total: 6.84ms	remaining: 395ms
7:	learn: 11.1256175	total: 7.78ms	remaining: 392ms
8:	learn: 9.9879755	total: 8.68ms	remaining: 388ms
9:	learn: 8.9822989	total: 9.56ms	remaining: 383ms
10:	learn: 8.1091006	total: 10.5ms	remaining: 382ms
11:	learn: 7.3231759	total: 11.4ms	remaining: 380ms
12:	learn: 6.6369256	total: 12.4ms	remaining: 379ms
13:	learn: 6.0247899	total: 13.3ms	remaining: 377ms
14:	learn: 5.4878199	total: 14.2ms	remaining: 375ms
15:	learn: 5.0123583	total: 15.1ms	remaining: 374ms
16:	learn: 4.5959167	total: 16.4ms	remaining: 379ms
17:	learn: 4.2321266	total: 17.3ms	remaining: 379ms
18:	learn: 3.8974303	total: 18.3ms	remaining: 377ms
19:	learn: 3.6

✅ Best params for CatBoost: {'iterations': 411, 'depth': 7, 'learning_rate': 0.11693909733978508, 'l2_leaf_reg': 1.0058943176207062, 'bagging_temperature': 0.7947457289044807, 'random_strength': 0.02570983831467845}


In [ ]:
params_Cat_20 ={'iterations': 411, 'depth': 7, 'learning_rate': 0.11693909733978508, 'l2_leaf_reg': 1.0058943176207062, 'bagging_temperature': 0.7947457289044807, 'random_strength': 0.02570983831467845}
Cat_20 = train_model(CatBoostRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_Cat_20)
df_test['Cat'] = Cat_20['y_pred']
df_train['Cat'] = Cat_20['model'].predict(Cat_20['X_train'])

0:	learn: 23.0829200	total: 59.9ms	remaining: 24.6s
1:	learn: 20.7107634	total: 61.8ms	remaining: 12.6s
2:	learn: 18.7012581	total: 73.7ms	remaining: 10s
3:	learn: 16.7208480	total: 75.4ms	remaining: 7.67s
4:	learn: 15.1583005	total: 87ms	remaining: 7.06s
5:	learn: 13.6169841	total: 89.3ms	remaining: 6.03s
6:	learn: 12.2274416	total: 91.4ms	remaining: 5.27s
7:	learn: 11.1256175	total: 93.6ms	remaining: 4.72s
8:	learn: 9.9879755	total: 107ms	remaining: 4.76s
9:	learn: 8.9822989	total: 115ms	remaining: 4.63s
10:	learn: 8.1091006	total: 126ms	remaining: 4.57s
11:	learn: 7.3231759	total: 133ms	remaining: 4.43s
12:	learn: 6.6369256	total: 139ms	remaining: 4.25s
13:	learn: 6.0247899	total: 149ms	remaining: 4.24s
14:	learn: 5.4878199	total: 152ms	remaining: 4.01s
15:	learn: 5.0123583	total: 162ms	remaining: 4.01s
16:	learn: 4.5959167	total: 175ms	remaining: 4.06s
17:	learn: 4.2321266	total: 177ms	remaining: 3.88s
18:	learn: 3.8974303	total: 182ms	remaining: 3.76s
19:	learn: 3.6054258	total: 1

In [ ]:
print(Cat_20['rmse'])
print(r2_score(Cat_20['y_test'],Cat_20['y_pred']))

0.9588097968974456
0.9988929346279354


### 75-25

In [ ]:
catb_25 = train_model(CatBoostRegressor,X,y,0.25,objective_catb,"XGBoost",n_trials=500)

[I 2025-06-10 12:18:27,432] A new study created in memory with name: no-name-1a2c656c-3eaa-4364-bfc3-9e9bf85860c7
[I 2025-06-10 12:18:32,659] Trial 0 finished with value: 4.7302891837547225 and parameters: {'iterations': 374, 'depth': 9, 'learning_rate': 0.0708667063364704, 'l2_leaf_reg': 9.42978528545916, 'bagging_temperature': 0.793247453037666, 'random_strength': 0.7976111763206573}. Best is trial 0 with value: 4.7302891837547225.
[I 2025-06-10 12:18:36,897] Trial 1 finished with value: 9.35401361215402 and parameters: {'iterations': 346, 'depth': 9, 'learning_rate': 0.006220937119958777, 'l2_leaf_reg': 3.7859169081364965, 'bagging_temperature': 0.6798532313233497, 'random_strength': 0.5118950383989619}. Best is trial 0 with value: 4.7302891837547225.
[I 2025-06-10 12:18:37,382] Trial 2 finished with value: 3.3145200636101415 and parameters: {'iterations': 221, 'depth': 4, 'learning_rate': 0.2821813055314162, 'l2_leaf_reg': 6.887658386785665, 'bagging_temperature': 0.810431966351, '

✅ Best params for XGBoost: {'iterations': 125, 'depth': 7, 'learning_rate': 0.2921363616393329, 'l2_leaf_reg': 2.030681574446381, 'bagging_temperature': 0.6483682865143723, 'random_strength': 0.05770675755307243}


0:	learn: 19.2177482	total: 1.6ms	remaining: 198ms
1:	learn: 15.0281148	total: 3.29ms	remaining: 203ms
2:	learn: 11.4816518	total: 4.55ms	remaining: 185ms
3:	learn: 8.9013278	total: 5.47ms	remaining: 166ms
4:	learn: 6.9904157	total: 6.47ms	remaining: 155ms
5:	learn: 5.5127445	total: 7.41ms	remaining: 147ms
6:	learn: 4.4821964	total: 8.3ms	remaining: 140ms
7:	learn: 3.7373025	total: 9.22ms	remaining: 135ms
8:	learn: 3.2154426	total: 10.2ms	remaining: 131ms
9:	learn: 2.8068861	total: 11ms	remaining: 127ms
10:	learn: 2.5259980	total: 12ms	remaining: 124ms
11:	learn: 2.3076069	total: 12.9ms	remaining: 122ms
12:	learn: 2.1350033	total: 13.8ms	remaining: 119ms
13:	learn: 1.9806837	total: 14.7ms	remaining: 117ms
14:	learn: 1.8590939	total: 15.6ms	remaining: 114ms
15:	learn: 1.7132791	total: 16.6ms	remaining: 113ms
16:	learn: 1.5925023	total: 17.4ms	remaining: 111ms
17:	learn: 1.4912920	total: 18.4ms	remaining: 109ms
18:	learn: 1.4258970	total: 19.3ms	remaining: 108ms
19:	learn: 1.3771200	tota

In [ ]:
print(catb_25['rmse'])
print(r2_score(catb_25['y_test'],catb_25['y_pred']))

3.27623581510089
0.9870514718188278


### 70-30

In [ ]:
catb_30 = train_model(CatBoostRegressor,X,y,0.3,objective_catb,"XGBoost",n_trials=500)

[I 2025-06-10 12:21:45,224] A new study created in memory with name: no-name-49beca93-d1c0-48dd-8288-b478a8c3f64d
[I 2025-06-10 12:21:45,517] Trial 0 finished with value: 21.232787279787807 and parameters: {'iterations': 395, 'depth': 6, 'learning_rate': 0.0015637487697724875, 'l2_leaf_reg': 8.832897888943112, 'bagging_temperature': 0.13004864005119998, 'random_strength': 0.539863184005383}. Best is trial 0 with value: 21.232787279787807.
[I 2025-06-10 12:21:45,718] Trial 1 finished with value: 4.41408262651192 and parameters: {'iterations': 152, 'depth': 7, 'learning_rate': 0.21313718719072505, 'l2_leaf_reg': 8.182146373925379, 'bagging_temperature': 0.8238568667201207, 'random_strength': 0.1589958696973669}. Best is trial 1 with value: 4.41408262651192.
[I 2025-06-10 12:21:46,850] Trial 2 finished with value: 3.5845252920588067 and parameters: {'iterations': 186, 'depth': 10, 'learning_rate': 0.12417252396543216, 'l2_leaf_reg': 7.59985538666899, 'bagging_temperature': 0.8206129479181

✅ Best params for XGBoost: {'iterations': 199, 'depth': 9, 'learning_rate': 0.22463852413069957, 'l2_leaf_reg': 9.603281945389135, 'bagging_temperature': 0.9897198453299537, 'random_strength': 0.037316499004809975}


0:	learn: 21.9396436	total: 7.13ms	remaining: 1.41s
1:	learn: 18.7009196	total: 11.9ms	remaining: 1.18s
2:	learn: 16.0810595	total: 14.9ms	remaining: 974ms
3:	learn: 14.1216159	total: 18.2ms	remaining: 887ms
4:	learn: 12.6342989	total: 21.1ms	remaining: 819ms
5:	learn: 11.3255835	total: 24ms	remaining: 772ms
6:	learn: 10.0929282	total: 26.9ms	remaining: 737ms
7:	learn: 9.0275419	total: 29.8ms	remaining: 710ms
8:	learn: 8.0489792	total: 32.5ms	remaining: 686ms
9:	learn: 7.2179132	total: 35.3ms	remaining: 667ms
10:	learn: 6.5995159	total: 38.1ms	remaining: 651ms
11:	learn: 5.9366638	total: 40.9ms	remaining: 638ms
12:	learn: 5.3560279	total: 43.7ms	remaining: 626ms
13:	learn: 4.8922407	total: 46.5ms	remaining: 615ms
14:	learn: 4.4338301	total: 49.5ms	remaining: 607ms
15:	learn: 4.0730696	total: 52.3ms	remaining: 599ms
16:	learn: 3.7533139	total: 55.1ms	remaining: 590ms
17:	learn: 3.4582140	total: 58ms	remaining: 583ms
18:	learn: 3.1982341	total: 64.5ms	remaining: 611ms
19:	learn: 2.978020

In [ ]:
print(catb_30['rmse'])
print(r2_score(catb_30['y_test'],catb_30['y_pred']))

3.330008838515967
0.987195424602216


### 60-40

In [ ]:
catb_40 = train_model(CatBoostRegressor,X,y,0.4,objective_catb,"XGBoost",n_trials=500)

[I 2025-06-10 12:29:14,162] A new study created in memory with name: no-name-bdf95d93-4a5d-4846-86b6-cc83a8ebcda0
[I 2025-06-10 12:29:14,381] Trial 0 finished with value: 4.683720446671024 and parameters: {'iterations': 115, 'depth': 8, 'learning_rate': 0.07470868043012187, 'l2_leaf_reg': 5.083569241260383, 'bagging_temperature': 0.6467968154572648, 'random_strength': 0.3478916668095997}. Best is trial 0 with value: 4.683720446671024.
[I 2025-06-10 12:29:16,307] Trial 1 finished with value: 22.967257346920462 and parameters: {'iterations': 415, 'depth': 10, 'learning_rate': 0.0011810423859260953, 'l2_leaf_reg': 5.453382790205356, 'bagging_temperature': 0.8672479968455473, 'random_strength': 0.43273762088365275}. Best is trial 0 with value: 4.683720446671024.
[I 2025-06-10 12:29:16,916] Trial 2 finished with value: 24.0555570398252 and parameters: {'iterations': 360, 'depth': 7, 'learning_rate': 0.0009828045976621102, 'l2_leaf_reg': 9.358444052094065, 'bagging_temperature': 0.9982269300

✅ Best params for XGBoost: {'iterations': 325, 'depth': 6, 'learning_rate': 0.2053504994359027, 'l2_leaf_reg': 3.947964767917329, 'bagging_temperature': 0.5404495815243666, 'random_strength': 0.16879122167848576}


0:	learn: 21.1036234	total: 721us	remaining: 234ms
1:	learn: 18.0171813	total: 1.33ms	remaining: 215ms
2:	learn: 15.6811989	total: 1.92ms	remaining: 206ms
3:	learn: 13.8187318	total: 2.57ms	remaining: 206ms
4:	learn: 12.0858288	total: 3.14ms	remaining: 201ms
5:	learn: 10.4036354	total: 3.71ms	remaining: 197ms
6:	learn: 9.0717821	total: 4.27ms	remaining: 194ms
7:	learn: 7.9970067	total: 4.88ms	remaining: 193ms
8:	learn: 7.0671794	total: 5.46ms	remaining: 192ms
9:	learn: 6.2477791	total: 6.01ms	remaining: 189ms
10:	learn: 5.6133421	total: 6.56ms	remaining: 187ms
11:	learn: 5.1157166	total: 7.13ms	remaining: 186ms
12:	learn: 4.6314970	total: 7.67ms	remaining: 184ms
13:	learn: 4.2315386	total: 8.24ms	remaining: 183ms
14:	learn: 3.9611270	total: 8.78ms	remaining: 182ms
15:	learn: 3.6723648	total: 9.33ms	remaining: 180ms
16:	learn: 3.4435085	total: 9.89ms	remaining: 179ms
17:	learn: 3.2033120	total: 10.4ms	remaining: 178ms
18:	learn: 3.0559634	total: 11.4ms	remaining: 184ms
19:	learn: 2.9082

In [ ]:
print(catb_40['rmse'])
print(r2_score(catb_40['y_test'],catb_40['y_pred']))

4.095075015703943
0.98013211122761


## XRT

In [ ]:
def objective_xrt(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = model_cls(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20


In [ ]:
xrt_20 = train_model(ExtraTreesRegressor,X,y,0.2,objective_xrt,"XRT",n_trials=300)

[I 2025-06-10 12:33:02,544] A new study created in memory with name: no-name-0d63d915-5299-4b23-a05e-ba5294497860
[I 2025-06-10 12:33:02,775] Trial 0 finished with value: 7.788476617853182 and parameters: {'n_estimators': 158, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 0 with value: 7.788476617853182.
[I 2025-06-10 12:33:03,087] Trial 1 finished with value: 9.498932145513423 and parameters: {'n_estimators': 282, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 10, 'max_features': None}. Best is trial 0 with value: 7.788476617853182.
[I 2025-06-10 12:33:03,213] Trial 2 finished with value: 10.296337957960844 and parameters: {'n_estimators': 103, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'log2'}. Best is trial 0 with value: 7.788476617853182.
[I 2025-06-10 12:33:03,679] Trial 3 finished with value: 6.184315222713072 and parameters: {'n_estimators': 438, 'max_depth': 9, 'min_samples_

✅ Best params for XRT: {'n_estimators': 147, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 2.2297


✅ Best params for XRT: {'n_estimators': 147, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


In [ ]:
params_XRT_20 = {'n_estimators': 147, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}
XRT_20 = train_model(ExtraTreesRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_XRT_20)
df_test['XRT'] = XRT_20['y_pred']
df_train['XRT'] = XRT_20['model'].predict(XRT_20['X_train'])

📊 CatBoost RMSE: 2.1074


In [ ]:
print(XRT_20['rmse'])
print(r2_score(XRT_20['y_test'],XRT_20['y_pred']))

2.1073562223576694
0.9946520880453034


In [ ]:
df_train

,AR,phi,L,QW,Qa,Va,h,R,sigma,gbm,xgb,Cat,XRT
75,0.600000,20,4,10.8,37.597795,23.64,1.8800,0.006970,-6.644326,-6.466626,-6.404846,-6.611249,-6.626059
442,1.000000,30,6,5.0,3.689801,2.32,7.6656,0.009108,17.974254,17.861558,17.861685,17.908610,17.977647
15,0.600000,10,4,1.9,4.500921,2.83,0.9400,0.004000,13.166125,13.216798,13.412960,13.192957,13.161131
325,1.666667,60,4,16.8,5.168902,3.25,3.2400,0.012050,5.919795,5.974513,5.624521,5.828181,5.933315
388,1.000000,15,6,5.6,24.333599,15.30,6.5610,0.006653,-3.789019,-3.788529,-3.817929,-3.877146,-3.801259
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.600000,30,6,5.0,0.636173,0.40,2.8200,0.009260,18.348362,18.526878,18.514587,18.380037,18.347884
270,1.666667,30,4,13.0,30.456759,19.15,1.6200,0.006910,-5.836333,-5.927690,-5.746362,-5.845778,-5.687657
348,1.000000,10,2,6.0,34.401029,21.63,5.8785,0.005543,-4.384864,-4.456202,-4.882215,-4.436144,-4.476300
435,1.000000,30,4,5.0,4.198739,2.64,7.6656,0.009108,17.717556,17.832721,17.727320,17.778539,17.734026


### 75-25

In [ ]:
xrt_25 = train_model(ExtraTreesRegressor,X,y,0.25,objective_xrt,"XRT",n_trials=300)

[I 2025-06-10 12:34:50,937] A new study created in memory with name: no-name-662c2575-4c2e-41a4-bcc8-ac568a4d7339
[I 2025-06-10 12:34:51,126] Trial 0 finished with value: 16.677487802327146 and parameters: {'n_estimators': 179, 'max_depth': 3, 'min_samples_split': 6, 'min_samples_leaf': 6, 'max_features': 'log2'}. Best is trial 0 with value: 16.677487802327146.
[I 2025-06-10 12:34:51,480] Trial 1 finished with value: 7.059186607183779 and parameters: {'n_estimators': 321, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 1 with value: 7.059186607183779.
[I 2025-06-10 12:34:51,827] Trial 2 finished with value: 14.653952813640855 and parameters: {'n_estimators': 340, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 1 with value: 7.059186607183779.
[I 2025-06-10 12:34:52,000] Trial 3 finished with value: 11.86996157705639 and parameters: {'n_estimators': 154, 'max_depth': 18, 'min_sample

✅ Best params for XRT: {'n_estimators': 151, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 2.5839


In [ ]:
print(xrt_25['rmse'])
print(r2_score(xrt_25['y_test'],xrt_25['y_pred']))

2.5839205359543485
0.9919456887918905


### 70-30

In [ ]:
xrt_30 = train_model(ExtraTreesRegressor,X,y,0.3,objective_xrt,"XRT",n_trials=300)

[I 2025-06-10 12:37:28,115] A new study created in memory with name: no-name-549ca8e9-3e5f-424c-a140-6921d6d8cfcd
[I 2025-06-10 12:37:28,602] Trial 0 finished with value: 4.96934258328089 and parameters: {'n_estimators': 461, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 4.96934258328089.
[I 2025-06-10 12:37:29,074] Trial 1 finished with value: 3.0988554037703437 and parameters: {'n_estimators': 417, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None}. Best is trial 1 with value: 3.0988554037703437.
[I 2025-06-10 12:37:29,481] Trial 2 finished with value: 14.240182281668435 and parameters: {'n_estimators': 374, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}. Best is trial 1 with value: 3.0988554037703437.
[I 2025-06-10 12:37:30,001] Trial 3 finished with value: 13.594226075755703 and parameters: {'n_estimators': 482, 'max_depth': 12, 'min_samples_

✅ Best params for XRT: {'n_estimators': 151, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 2.5978


In [ ]:
print(xrt_30['rmse'])
print(r2_score(xrt_30['y_test'],xrt_30['y_pred']))

2.5978374114532206
0.9922071136449876


### 60-40

In [ ]:
xrt_40 = train_model(ExtraTreesRegressor,X,y,0.4,objective_xrt,"XRT",n_trials=300)

[I 2025-06-10 12:39:18,536] A new study created in memory with name: no-name-bde660b5-0b99-4855-b1ef-425118eb622a
[I 2025-06-10 12:39:18,858] Trial 0 finished with value: 13.33895425462392 and parameters: {'n_estimators': 270, 'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 0 with value: 13.33895425462392.
[I 2025-06-10 12:39:19,050] Trial 1 finished with value: 10.32880638620009 and parameters: {'n_estimators': 158, 'max_depth': 5, 'min_samples_split': 7, 'min_samples_leaf': 7, 'max_features': None}. Best is trial 1 with value: 10.32880638620009.
[I 2025-06-10 12:39:19,587] Trial 2 finished with value: 13.681168575705623 and parameters: {'n_estimators': 494, 'max_depth': 7, 'min_samples_split': 9, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 with value: 10.32880638620009.
[I 2025-06-10 12:39:19,824] Trial 3 finished with value: 11.233354736024907 and parameters: {'n_estimators': 191, 'max_depth': 14, 'min_sample

✅ Best params for XRT: {'n_estimators': 156, 'max_depth': 17, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None}


📊 XRT RMSE: 3.1942


In [ ]:
print(xrt_40['rmse'])
print(r2_score(xrt_40['y_test'],xrt_40['y_pred']))

3.1942410271716035
0.9879117519419459


## Random Forest

In [ ]:
def objective_rf(trial, X_train, y_train, X_test, y_test,model_cls):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
    }

    model = model_cls(**params, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    return rmse


### 80-20

In [ ]:
rf_20 = train_model(RandomForestRegressor,X,y,0.2,objective_rf,"RF",n_trials=300)

[I 2025-06-10 12:43:10,880] A new study created in memory with name: no-name-a9397b8c-0a24-410a-86a8-b4686db9917d
[I 2025-06-10 12:43:11,385] Trial 0 finished with value: 5.7557971819318805 and parameters: {'n_estimators': 316, 'max_depth': 11, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt'}. Best is trial 0 with value: 5.7557971819318805.
[I 2025-06-10 12:43:11,716] Trial 1 finished with value: 7.159639747587734 and parameters: {'n_estimators': 195, 'max_depth': 11, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 0 with value: 5.7557971819318805.
[I 2025-06-10 12:43:12,123] Trial 2 finished with value: 8.610797476634467 and parameters: {'n_estimators': 257, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 9, 'max_features': 'log2'}. Best is trial 0 with value: 5.7557971819318805.
[I 2025-06-10 12:43:12,830] Trial 3 finished with value: 7.527394835446548 and parameters: {'n_estimators': 475, 'max_depth': 3, 'min_sa

✅ Best params for RF: {'n_estimators': 489, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 1.9241


✅ Best params for RF: {'n_estimators': 489, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None}


In [ ]:
params_RF_20 = {'n_estimators': 489, 'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None}
RF_20 = train_model(RandomForestRegressor,X,y,0.2,None,"CatBoost",n_trials=0,**params_RF_20)
df_test['RF'] = RF_20['y_pred']
df_train['RF'] = RF_20['model'].predict(RF_20['X_train'])

📊 CatBoost RMSE: 1.9500


In [ ]:
print(RF_20['rmse'])
print(r2_score(RF_20['y_test'],RF_20['y_pred']))

1.9500005443256927
0.9954209244811444


In [ ]:
np.savetxt('training.csv',df_train,delimiter=',')
np.savetxt('testing.csv',df_test,delimiter=',')

### 75-25

In [ ]:
rf_25 = train_model(RandomForestRegressor,X,y,0.25,objective_rf,"RF",n_trials=300)

[I 2025-06-10 12:47:30,120] A new study created in memory with name: no-name-fd5f79c2-36c4-4ebd-8ed9-37a384837d72
[I 2025-06-10 12:47:30,889] Trial 0 finished with value: 5.8378081946012506 and parameters: {'n_estimators': 460, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_features': None}. Best is trial 0 with value: 5.8378081946012506.
[I 2025-06-10 12:47:31,665] Trial 1 finished with value: 5.163515149145118 and parameters: {'n_estimators': 470, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': 'log2'}. Best is trial 1 with value: 5.163515149145118.
[I 2025-06-10 12:47:32,463] Trial 2 finished with value: 9.107383825151663 and parameters: {'n_estimators': 449, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 7, 'max_features': 'sqrt'}. Best is trial 1 with value: 5.163515149145118.
[I 2025-06-10 12:47:33,250] Trial 3 finished with value: 8.87680682327493 and parameters: {'n_estimators': 470, 'max_depth': 4, 'min_samples

✅ Best params for RF: {'n_estimators': 115, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2'}


📊 RF RMSE: 4.7685


In [ ]:
print(rf_25['rmse'])
print(r2_score(rf_25['y_test'],rf_25['y_pred']))

4.76846615010347
0.9725698926541329


### 70-30

In [ ]:
rf_30 = train_model(RandomForestRegressor,X,y,0.3,objective_rf,"RF",n_trials=300)

[I 2025-06-10 12:49:20,282] A new study created in memory with name: no-name-bf7d01a3-1810-4850-b8c4-d63077d9fa4a
[I 2025-06-10 12:49:20,488] Trial 0 finished with value: 5.070697186619027 and parameters: {'n_estimators': 103, 'max_depth': 16, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 0 with value: 5.070697186619027.
[I 2025-06-10 12:49:20,996] Trial 1 finished with value: 6.476264084491837 and parameters: {'n_estimators': 336, 'max_depth': 4, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': None}. Best is trial 0 with value: 5.070697186619027.
[I 2025-06-10 12:49:21,201] Trial 2 finished with value: 5.779620665708082 and parameters: {'n_estimators': 122, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 5.070697186619027.
[I 2025-06-10 12:49:21,609] Trial 3 finished with value: 6.995712737882937 and parameters: {'n_estimators': 269, 'max_depth': 9, 'min_samples_spli

✅ Best params for RF: {'n_estimators': 326, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2'}


📊 RF RMSE: 4.2942


In [ ]:
print(rf_30['rmse'])
print(r2_score(rf_30['y_test'],rf_30['y_pred']))

4.294155393031729
0.9787073398797802


### 60-40

In [ ]:
rf_40 = train_model(RandomForestRegressor,X,y,0.4,objective_rf,"RF",n_trials=300)

[I 2025-06-10 12:52:14,263] A new study created in memory with name: no-name-6716f5bb-771a-46a9-b2a7-9655d7d8a84f
[I 2025-06-10 12:52:14,538] Trial 0 finished with value: 5.192630816694647 and parameters: {'n_estimators': 146, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 5.192630816694647.
[I 2025-06-10 12:52:15,338] Trial 1 finished with value: 11.978088183809874 and parameters: {'n_estimators': 361, 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 10, 'max_features': 'sqrt'}. Best is trial 0 with value: 5.192630816694647.
[I 2025-06-10 12:52:16,023] Trial 2 finished with value: 6.982534607732143 and parameters: {'n_estimators': 201, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 6, 'max_features': None}. Best is trial 0 with value: 5.192630816694647.
[I 2025-06-10 12:52:16,564] Trial 3 finished with value: 8.028760369758155 and parameters: {'n_estimators': 267, 'max_depth': 18, 'min_samples_

✅ Best params for RF: {'n_estimators': 322, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': None}


📊 RF RMSE: 4.4730


In [ ]:
print(rf_40['rmse'])
print(r2_score(rf_40['y_test'],rf_40['y_pred']))

4.473012258954106
0.976295643326699


## Saving Files

In [ ]:
df_test


,AR,phi,L,QW,Qa,Va,h,R,sigma,gbm,xgb,Cat,XRT,RF
173,1.666667,10,1,5.7,23.872374,15.01,0.5400,0.002550,-1.466537,-2.075985,-2.512376,-1.922327,-2.738027,-2.507775
274,1.666667,30,6,5.2,1.351867,0.85,1.6200,0.006910,16.882085,16.205871,13.090716,14.377220,15.238124,12.592974
489,1.000000,60,2,29.0,24.667589,15.51,7.9886,0.012535,-2.385587,-3.219356,-3.154209,-2.681359,-3.461905,-3.246578
72,0.600000,20,4,5.4,12.262225,7.71,1.8800,0.006970,3.429377,3.275333,3.017687,3.359856,3.163236,2.968260
305,1.666667,40,6,16.0,10.894454,6.85,2.1600,0.008780,1.905303,2.414572,2.493150,2.142582,2.488356,2.501290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,1.666667,60,6,12.6,0.000000,0.00,3.2400,0.012050,11.573187,11.096997,14.989950,12.665798,12.046644,10.152084
411,1.000000,20,4,10.8,41.732917,26.24,7.0444,0.007577,-4.956412,-5.087159,-5.322308,-5.134345,-5.926183,-6.350960
502,1.000000,60,6,25.2,8.890511,5.59,7.9886,0.012535,2.171858,3.158511,4.944877,2.989816,4.373929,4.888263
349,1.000000,10,2,7.1,40.842275,25.68,5.8785,0.005543,-4.839180,-6.881480,-6.631685,-6.456282,-6.776738,-6.966650


In [ ]:
df_train

,AR,phi,L,QW,Qa,Va,h,R,sigma,gbm,xgb,Cat,XRT,RF
75,0.600000,20,4,10.8,37.597795,23.64,1.8800,0.006970,-6.644326,-6.466626,-6.404846,-6.611249,-6.626059,-6.300201
442,1.000000,30,6,5.0,3.689801,2.32,7.6656,0.009108,17.974254,17.861558,17.861685,17.908610,17.977647,18.055666
15,0.600000,10,4,1.9,4.500921,2.83,0.9400,0.004000,13.166125,13.216798,13.412960,13.192957,13.161131,13.119601
325,1.666667,60,4,16.8,5.168902,3.25,3.2400,0.012050,5.919795,5.974513,5.624521,5.828181,5.933315,5.638843
388,1.000000,15,6,5.6,24.333599,15.30,6.5610,0.006653,-3.789019,-3.788529,-3.817929,-3.877146,-3.801259,-3.576849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,0.600000,30,6,5.0,0.636173,0.40,2.8200,0.009260,18.348362,18.526878,18.514587,18.380037,18.347884,18.451602
270,1.666667,30,4,13.0,30.456759,19.15,1.6200,0.006910,-5.836333,-5.927690,-5.746362,-5.845778,-5.687657,-5.076516
348,1.000000,10,2,6.0,34.401029,21.63,5.8785,0.005543,-4.384864,-4.456202,-4.882215,-4.436144,-4.476300,-5.218813
435,1.000000,30,4,5.0,4.198739,2.64,7.6656,0.009108,17.717556,17.832721,17.727320,17.778539,17.734026,18.050762


In [ ]:
df_test.to_csv("dim_testing.csv",index=False)
df_train.to_csv("dim_training.csv",index = False)